In [1]:
#generate for bar chart
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import word2vec
import json
import numpy as np
from konlpy.tag import Kkma
from konlpy.utils import pprint
import math

l_quality= open("lecture_quality.txt", "r", encoding="utf-8").readlines()
load= open("load.txt", "r", encoding="utf-8").readlines()
grade= open("grade.txt", "r", encoding="utf-8").readlines()
int_friendly= open("international_friendly.txt", "r", encoding="utf-8").readlines()
CATEGORY_COUNT = 3

C:\Users\kvtor\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
pos_vectors = KeyedVectors.load_word2vec_format('../pos_naver_news.vec', binary=False)

In [2]:
scores = []
subjects = [l_quality, load, grade]
def add_to_dict(subject):
    subject_scores={}
    for i in range(len(subject)):
        p = subject[i].split("\t")
        subject_scores[p[0]] = float(p[1][:-1])
    scores.append(subject_scores)

for subject in subjects:
    add_to_dict(subject)

In [3]:
subject_mp = {"강의":0,"널널":1,"성적":2}
def calculate_score(s, subject_str):
    result = 0
    for k,v in scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

In [4]:
REVIEWS = "cs_reviews"

f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

scores_list = []
subjects_str = ["강의", "널널", "성적"]
for subject_str in subjects_str:
    scores_list_subject=[]
    for i in range(len(f[:])):
        line = f[i]
        course = json.loads(line)
        scores_list_subject.append(calculate_score(course["review_text"], subject_str))
    scores_list.append(scores_list_subject)


In [5]:
f_count_word = {}
count_word = {}
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}

def calc_weight(s):
    return 3 - (mp[s])

for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] == "?":
        continue
    words = course["review_text"].split()
    for w in words:
        if w not in count_word:
            count_word[w] = 0
        count_word[w] += calc_weight(course["강의"])


In [6]:
sorted_by_value = sorted(count_word.items(), key=lambda kv: kv[1])[::-1]

In [7]:
# sorted_by_value

[('전혀', 15),
 ('저희가', 12),
 ('디자인', 11),
 ('애들이', 8),
 ('조교님께', 8),
 ('못하십니다.교수님이', 8),
 ('학생에', 7),
 ('너무나', 7),
 ('일', 6),
 ('핀토스', 6),
 ('숙제를', 6),
 ('특허', 6),
 ('제발', 5),
 ('조교가', 5),
 ('족보', 5),
 ('나오기', 5),
 ('정작', 5),
 ('학교', 5),
 ('틀린', 5),
 ('제일', 5),
 ('일부', 5),
 ('잘했냐에', 4),
 ('제시간에', 4),
 ('돼', 4),
 ('이해와', 4),
 ('디자인을', 4),
 ('부적절하고', 4),
 ('받는데,', 4),
 ('많은걸', 4),
 ('필기체를', 4),
 ('책이나', 4),
 ('애니메이션', 4),
 ('관리', 4),
 ('학교에서', 4),
 ('최악의', 4),
 ('꿀이나', 4),
 ('이야기를', 4),
 ('질은', 4),
 ('즉', 4),
 ('절대로', 4),
 ('매시간', 4),
 ('차례', 4),
 ('오토마타를', 4),
 ('그런거', 4),
 ('강의계획서에', 4),
 ('들어갑니다.', 4),
 ('명세서', 4),
 ('귀찮습니다.', 3),
 ('예상하지', 3),
 ('곳에서', 3),
 ('입장은', 3),
 ('설계,', 3),
 ('돌고', 3),
 ('Agile을', 3),
 ('퍼포먼스', 3),
 ('좋겠음.', 3),
 ('이번학기', 3),
 ('하셔도', 3),
 ('글을', 3),
 ('됐음.', 3),
 ('글씨', 3),
 ('적힌', 3),
 ('맞고', 3),
 ('Agile', 3),
 ('수준은', 3),
 ('결코', 3),
 ('주시네요', 3),
 ('강의에', 3),
 ('적습니다.', 3),
 ('땅에', 3),
 ('눈', 3),
 ('"이', 3),
 ('내용으로', 3),
 ('오타도', 3),
 ('좋은데', 3),
 ('얼마나'

In [16]:
def calc_score_avg(column_name,comment_id):
    i = comment_id
    course = json.loads(f[i])
    #print(mp[course["강의"]],(scores_list[i]))
    return scores_list[subject_mp[column_name]][i]
    

for i in range(len(f)):
    course = json.loads(f[i])
    print(course["course_code"])
    
    for x in subjects_str:
        print(x,calc_score_avg(x,i))
    break
    

CS322
강의 0.0996679946249559
널널 -0.761594155955765
성적 -0.8336546070121552


In [19]:
def append_noun(s):
    return "('%s','Noun')" % (s)

naver_wv_categories = ["강의", "부담", "성적"]
wv_update = {}
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        if append_noun(k) in pos_vectors.vocab:
#             print(k,naver_wv_categories[i])
#             print(pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i])))
            wv_update[k] = pos_vectors.similarity(append_noun(k),append_noun(naver_wv_categories[i]))
        

In [20]:
review_update = {}
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)

    for i in range(CATEGORY_COUNT):
        if course[subjects_str[i]] == "?":
            continue
        for k,v in scores[i].items():
            if k in course["review_text"].split():
                if k not in review_update:
                    review_update[k] = 0
                review_update[k] += calc_weight(course[subjects_str[i]])*100.0/len(course["review_text"])
        

In [21]:
aggregated_scores = [{} for i in range(CATEGORY_COUNT)]
for i in range(CATEGORY_COUNT):
    for k,v in scores[i].items():
        aggregated_scores[i][k] = v
        if k in review_update:
            aggregated_scores[i][k] += review_update[k]*0.1
        
        if k in wv_update:
            aggregated_scores[i][k] += wv_update[k]
            
            

In [ ]:
def calc_aggregated_score(s, subject_str):
    result = 0
    for k,v in aggregated_scores[subject_mp[subject_str]].items():
        if k in s:
            result += v
#     result=1.0/(1+math.exp(-result))
    result=math.tanh(result)
    return result

def calc_aggregated_score_sum(column_name,i):
    course = json.loads(f[i])
    return calc_aggregated_score(course["review_text"], column_name)

# print(subjects_str)

In [35]:
with open("bar_chart_data.txt","w",encoding="utf-8") as out:
    for i in range(len(f)):
        course = json.loads(f[i])
    #     print(course["review_text"])
        out.write(course["course_code"]+"\t")
#         print(course["course_code"]+"\t",end="")
        for x in subjects_str:
            out.write(str(calc_aggregated_score_sum(x,i))+"\t")
#             print(str(calc_aggregated_score_sum(x,i))+"\t",end="")
        out.write("\n")